Import dependencies

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import itertools

Initialize empty dataframe

In [4]:
df = pd.DataFrame(columns=["title","address","num","reviews","rate"])

Scrape yellow pages with category "Real Estate Brokers", "Commercial Properties", or "Residential Properties" with different location from Luzon Visayas, and Mindanao from page 1 to 8

Data scraped: "title","address","num","reviews","rate"

In [5]:
category = ["Real Estate Brokers", "Commercial Properties", "Residential Properties"]
pages = range(8)
permutations = list(itertools.product(category, pages))

for cat,pg in permutations:
  url = "https://www.yellow-pages.ph/search/"+cat+"/nationwide/page-"+str(pg)
  req = requests.get(url)
  soup = BeautifulSoup(req.text, 'html.parser')

  links = soup.select('.search-tradename a')
  res_link = [link["href"] for link in links]
  res_title =[link.text for link in links]

  for i in range(len(res_link)):
    ins = []
    page_req = requests.get("https://www.yellow-pages.ph/"+res_link[i])
    soup = BeautifulSoup(page_req.text, 'html.parser')

    ins.append(res_title[i])#title

    try:#address
      ins.append(soup.select_one('.biz-link').text)
    except AttributeError:
      ins.append(None)


    try:#address
      ins.append([num.text for num in soup.select('.phn-txt')])
    except AttributeError:
      ins.append(None)

    try:#reviews
      ins.append([review.find('p').text for review in soup.find_all("div", class_="yp-see-morex text-break")])
    except AttributeError:
      ins.append(None)
      
    try:#rate
     ins.append(soup.select_one('.rating-num').text.strip())
    except AttributeError:
      ins.append(None)

    
    df.loc[len(df)] = ins

In [6]:
df

,title,address,num,reviews,rate
0,Luxury Manila,Mayor Gil Fernando Avenue corner Dragon Street...,"[+63 (2) 8 701 0765, +63 (2) 8 701 0765]",[],0
1,Mlabueva Real Estate Ventures,"New Frontier, Mabolo, Cebu City 6000 Cebu","[+63 (032) 513 2318, +63 (922) 209 3691, +63 (...",[],0
2,Propertymart.Ph,"Level 12, Robinsons Summit Center, 6783, Ayala...","[+63 (2) 8 736 0356, +63 (917) 891 1201, +63 (...",[],0
3,Green Circle Realty Sales,"902-B, West Tower, Philippine Stock Exchange C...","[+63 (2) 8 942 1184, +63 (948) 758 5315, +63 (...","[Reliable Agents and affordable Units!, Green ...",5.0
4,Gloria Go Pe Apartment Lessor,"1011 Apacible Street, Ermita, City of Manila, ...","[+63 (2) 7 218 7771, +63 (977) 803 0541, +63 (...",[],0
...,...,...,...,...,...
75,Aurelio C. Dela Pena,None,[],[],None
76,Lilibeth M. Macaso,None,[],[],None
77,Avelino S. Padilla,None,[],[],None
78,Feliza M. Ronquillo,None,[],[],None


Clean the dataframe by dropping instances with null values.

In [9]:
df = df.dropna()

In [10]:
df

,title,address,num,reviews,rate
0,Luxury Manila,Mayor Gil Fernando Avenue corner Dragon Street...,"[+63 (2) 8 701 0765, +63 (2) 8 701 0765]",[],0
1,Mlabueva Real Estate Ventures,"New Frontier, Mabolo, Cebu City 6000 Cebu","[+63 (032) 513 2318, +63 (922) 209 3691, +63 (...",[],0
2,Propertymart.Ph,"Level 12, Robinsons Summit Center, 6783, Ayala...","[+63 (2) 8 736 0356, +63 (917) 891 1201, +63 (...",[],0
3,Green Circle Realty Sales,"902-B, West Tower, Philippine Stock Exchange C...","[+63 (2) 8 942 1184, +63 (948) 758 5315, +63 (...","[Reliable Agents and affordable Units!, Green ...",5.0
4,Gloria Go Pe Apartment Lessor,"1011 Apacible Street, Ermita, City of Manila, ...","[+63 (2) 7 218 7771, +63 (977) 803 0541, +63 (...",[],0
...,...,...,...,...,...
65,Regino M. Gonzalez,"14 Hernadez Street, B. Pag Asa Subdivision, Ba...","[+63 (918) 924 7966, +63 (918) 924 7966]",[],0
66,Danilo M. Griarte,"B5 L16 Sarreal Village, Cavite City 4100 Cavite","[+63 (921) 408 7802, +63 (921) 408 7802]",[],0
67,Edmun E. Gumpay,"2/F, T. Decastro Building, Aguinaldo Highway,...","[+63 (920) 411 9060, +63 (46) 471 1726, +63 (9...",[],0
68,Reynaldo L. Hapa,"B L2-A Plaridel Subdivision, Bayan Luma Vii, I...","[+63 (46) 472 0292, +63 (917) 502 2861, +63 (4...",[],0


Save the dataset as csv

In [12]:
df.to_csv('data.csv', index=False)